https://square-firefly-0750.ploomberapp.io/api/dashboard_for_yandex_afisha/

In [102]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

Чтобы посмотреть, какие таблицы есть в базе данных, вы можете использовать SQL-запрос для получения списка таблиц. В PostgreSQL это можно сделать с помощью запроса к системной таблице `information_schema.tables`.

In [103]:
# Запрос для получения списка таблиц
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

In [104]:
tables = pd.read_sql(query, engine)
tables

,table_name
0,dash_visits
1,dash_visits_new
2,log_raw


In [111]:
table = pd.read_sql('select * from dash_visits', engine)
table.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [106]:
table.groupby(['dt', 'item_topic'])[['visits']].sum()

visits
dt                  item_topic                
2019-09-24 18:28:00 Деньги                  24
                    Дети                    32
                    Женская психология      19
                    Женщины                 27
                    Здоровье                25
...                                        ...
2019-09-24 19:00:00 Семья                  681
                    Скандалы               507
                    Туризм                 538
                    Шоу                    408
                    Юмор                   504

[425 rows x 1 columns]

In [107]:
(table.groupby('source_topic')[['visits']].sum() *100 / table.visits.sum()).sort_values('visits', ascending=False).head()

,visits
source_topic,
Семейные отношения,10.737669
Россия,9.616482
Полезные советы,8.836680
Путешествия,7.776743
Знаменитости,7.719039


In [108]:
top_item_topics = table.groupby('item_topic')['visits'].sum().sort_values(ascending=False).head(10).index.values
table['item_topic'] = table['item_topic'].apply(lambda x: x if x in top_item_topics else 'Другое')
top_item_topics

array(['Наука', 'Отношения', 'Интересные факты', 'Общество', 'Подборки',
       'Россия', 'Полезные советы', 'История', 'Семья', 'Женщины'],
      dtype=object)

In [ ]:
table.pivot_table(index = 'item_topic', columns = 'source_topic', values='visits', aggfunc='sum').fillna(0).astype(int)